# Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import os
import gc
from time import time

seed = 42

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Helper Functions

In [ ]:
def breaker():
    print("\n" + 50*"-" + "\n")
    
def head(x=None, no_of_ele=5):
    print(x[:no_of_ele])
    
def getCol(x=None):
    return [col for col in x.columns]

def getObjCol(x=None):
    s = (x.dtypes == "object")
    return list(s[s].index)

sc = StandardScaler()
le = LabelEncoder()

# Data Handling

In [ ]:
tr_set = pd.read_csv("../input/tabular-playground-series-mar-2021/train.csv")
ts_set = pd.read_csv("../input/tabular-playground-series-mar-2021/test.csv")

breaker()
print("Train Set Shape : {}".format(tr_set.shape))
breaker()
print("Test Set Shape  : {}".format(ts_set.shape))
breaker()

labels = tr_set["target"].copy().values

# Dropping categorical feature 10 due to KeyError when transforming test set
tr_set = tr_set.drop(labels=["id", "target", "cat10"], axis=1)
ts_set = ts_set.drop(labels=["id", "cat10"], axis=1)

tr_set = tr_set.copy().values
ts_set = ts_set.copy().values

Label Encoding, Scaling, Splitting

In [ ]:
for i in range(18):
    tr_set[:, i] = le.fit_transform(tr_set[:, i])
    ts_set[:, i] = le.transform(ts_set[:, i])
    
tr_set = sc.fit_transform(tr_set)
X_test = sc.transform(ts_set)

X_train, X_valid, y_train, y_valid = train_test_split(tr_set, labels, test_size=0.2,
                                                      shuffle=True, random_state=seed)

del tr_set, ts_set

breaker()
print("Garbage Collected : {}".format(gc.collect()))
breaker()

**Dataset Template**

In [ ]:
class DS(Dataset):
    def __init__(this, X=None, y=None, mode="train"):
        this.mode = mode
        this.X = X
        if mode == "train" or mode == "valid":
            this.y = y
            
    def __len__(this):
        return this.X.shape[0]

    def __getitem__(this, idx):
        if this.mode == "train" or this.mode == "valid":
            return torch.FloatTensor(this.X[idx]), torch.FloatTensor(this.y[idx])
        else:
            return torch.FloatTensor(this.X[idx])

# ANN Configuration and Setup

**Config**

In [ ]:
class CFG():
    tr_batch_size = 256
    va_batch_size = 256
    ts_batch_size = 256
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    OL = 1
    
    def __init__(this, IL=None, HL=None, DP=None, epochs=None, n_folds=None):
        this.IL = IL
        this.HL = HL
        this.DP = DP
        this.epochs = epochs
        this.n_folds = n_folds

**Setup**

In [ ]:
class ANN(nn.Module):
    def __init__(this, IL=None, HL=None, OL=None, use_DP=True, DP=0.5):

        super(ANN, this).__init__()

        this.use_DP = use_DP
        this.HL = HL
        if use_DP:
            this.DP_ = nn.Dropout(p=DP)
        
        if len(HL) == 1:
            this.BN1 = nn.BatchNorm1d(num_features=IL, eps=1e-5)
            this.FC1 = nn.Linear(in_features=IL, out_features=HL[0])

            this.BN2 = nn.BatchNorm1d(num_features=HL[0], eps=1e-5)
            this.FC2 = nn.Linear(in_features=HL[0], out_features=OL)
        
        elif len(HL) == 2:
            this.BN1 = nn.BatchNorm1d(num_features=IL, eps=1e-5)
            this.FC1 = nn.Linear(in_features=IL, out_features=HL[0])

            this.BN2 = nn.BatchNorm1d(num_features=HL[0], eps=1e-5)
            this.FC2 = nn.Linear(in_features=HL[0], out_features=HL[1])

            this.BN3 = nn.BatchNorm1d(num_features=HL[1], eps=1e-5)
            this.FC3 = nn.Linear(in_features=HL[1], out_features=OL)
        
        elif len(HL) == 3:
            this.BN1 = nn.BatchNorm1d(num_features=IL, eps=1e-5)
            this.FC1 = nn.Linear(in_features=IL, out_features=HL[0])

            this.BN2 = nn.BatchNorm1d(num_features=HL[0], eps=1e-5)
            this.FC2 = nn.Linear(in_features=HL[0], out_features=HL[1])

            this.BN3 = nn.BatchNorm1d(num_features=HL[1], eps=1e-5)
            this.FC3 = nn.Linear(in_features=HL[1], out_features=HL[2])

            this.BN4 = nn.BatchNorm1d(num_features=HL[2], eps=1e-5)
            this.FC4 = nn.Linear(in_features=HL[2], out_features=OL)

    def getOptimizer(this, A_S=True, lr=1e-3, wd=0):
        if A_S:
            return optim.Adam(this.parameters(), lr=lr, weight_decay=wd)
        else:
            return optim.SGD(this.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
    
    def getPlateauLR(this, optimizer=None, patience=None, eps=None):
        return optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=patience, eps=eps, verbose=True)
    
    def forward(this, x):
        if not this.use_DP:
            if len(this.HL) == 1:
                x = F.relu(this.FC1(this.BN1(x)))
                x = this.FC2(this.BN2(x))
                return x

            elif len(this.HL) == 2:
                x = F.relu(this.FC1(this.BN1(x)))
                x = F.relu(this.FC2(this.BN2(x)))
                x = this.FC3(this.BN3(x))
                return x

            elif len(this.HL) == 3:
                x = F.relu(this.FC1(this.BN1(x)))
                x = F.relu(this.FC2(this.BN2(x)))
                x = F.relu(this.FC3(this.BN3(x)))
                x = this.FC4(this.BN4(x))
                return x

        else:
            if len(this.HL) == 1:
                x = F.relu(this.DP_(this.FC1(this.BN1(x))))
                x = this.FC2(this.BN2(x))
                return x

            elif len(this.HL) == 1:
                x = F.relu(this.DP_(this.FC1(this.BN1(x))))
                x = F.relu(this.DP_(this.FC2(this.BN2(x))))
                x = this.FC3(this.BN3(x))
                return x

            elif len(this.HL) == 3:
                x = F.relu(this.DP_(this.FC1(this.BN1(x))))
                x = F.relu(this.DP_(this.FC2(this.BN2(x))))
                x = F.relu(this.DP_(this.FC3(this.BN3(x))))
                x = this.FC4(this.BN4(x))
                return x

**ANN Helpers**

In [ ]:
def fit_(model=None, optimizer=None, scheduler=None, epochs=None, 
         trainloader=None, validloader=None, 
         criterion=None, device=None, verbose=True, path=None):
    
    breaker()
    print("Training ...")
    breaker()

    model.to(device)

    Losses = []
    AUCs = []

    bestLoss = {"train" : np.inf, "valid" : np.inf}
    bestAUCs = {"train" : 0.0, "valid" : 0.0}
    DLS = {"train" : trainloader, "valid" : validloader}

    start_time = time()
    for e in range(epochs):
        e_st = time()

        epochLoss = {"train" : 0.0, "valid" : 0.0}
        epochAUCs = {"train" : 0.0, "valid" : 0.0}

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
            
            lossPerPass = []
            aucsPerPass = []

            for X, y in DLS[phase]:
                X, y = X.to(device), y.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    output = model(X)
                    loss = criterion(output, y)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                lossPerPass.append(loss.item())
                aucsPerPass.append(getAUC(y, output))
            epochLoss[phase] = np.mean(np.array(lossPerPass))
            epochAUCs[phase] = np.mean(np.array(aucsPerPass))
        Losses.append(epochLoss)
        AUCs.append(epochAUCs)

        torch.save({"model_state_dict" : model.state_dict(),
                    "optim_state_dict" : optimizer.state_dict()},
                   "./Epoch_{}.pt".format(e+1))
        
        if epochLoss["valid"] < bestLoss["valid"]:
            bestLoss = epochLoss
            bestLossEpoch = e+1
        
        if epochAUCs["valid"] > bestAUCs["valid"]:
            bestAUCs = epochAUCs
            bestAUCsEpoch = e+1
        
        if scheduler:
            scheduler.step(epochLoss["valid"])
        
        if verbose:
            print("Epoch : {} | Train Loss : {:.5f} | Valid Loss : {:.5f} \
| Train AUC : {:.5f} | Valid AUC : {:.5f} | Time : {:.2f} seconds".format(e + 1, epochLoss["train"], epochLoss["valid"],
                                                                                 epochAUCs["train"], epochAUCs["valid"],
                                                                                 time() - e_st))

    breaker()
    print("Best Validation Loss at Epoch {}".format(bestLossEpoch))   
    breaker()
    print("Best Validation AUCs at Epoch {}".format(bestAUCsEpoch))  
    breaker()
    print("Time Taken [{} Epochs] : {:.2f} seconds".format(epochs, (time() - start_time) / 60))
    breaker()
    print("Training Complete")
    breaker()

    return Losses, AUCs, bestLossEpoch, bestAUCsEpoch

def predict_(model=None, dataloader=None, device=None, mode="valid", path=None):
    if path:
        model.load_state_dict(torch.load(path)["model_state_dict"])
    
    model.to(device)
    model.eval()

    y_pred = torch.zeros(1, 1).to(device)

    if mode == "valid":
        for X, y in dataloader:
            X = X.to(device)
            with torch.no_grad():
                output = torch.sigmoid(model(X))
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    elif mode == "test":
        for X in dataloader:
            X = X.to(device)
            with torch.no_grad():
                output = torch.sigmoid(model(X))
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    else:
        print("Invalid Mode")
    
    return y_pred[1:].detach().cpu().numpy()

def getAUC(y_true=None, y_pred=None):
    y_true = y_true.detach().cpu().numpy()
    y_pred = torch.sigmoid(y_pred).detach().cpu().numpy()

    return roc_auc_score(y_true, y_pred)

**Training**

In [ ]:
cfg = CFG(IL=X_train.shape[1], HL=[16], epochs=50)

tr_data_setup = DS(X=X_train, y=y_train.reshape(-1, 1), mode="train")
va_data_setup = DS(X=X_valid, y=y_valid.reshape(-1, 1), mode="valid")
ts_data_setup = DS(X=X_test, y=None, mode="test")

tr_data = DL(tr_data_setup, batch_size=cfg.tr_batch_size, shuffle=True, generator=torch.manual_seed(seed), drop_last=True)
va_data = DL(va_data_setup, batch_size=cfg.va_batch_size, shuffle=False)
ts_data = DL(ts_data_setup, batch_size=cfg.ts_batch_size, shuffle=False)

torch.manual_seed(seed)
model = ANN(IL=cfg.IL, HL=cfg.HL, OL=cfg.OL, use_DP=False)
optimizer = model.getOptimizer(A_S=True, lr=1e-3, wd=1e-5)
# scheduler = model.getPlateauLR(optimizer=optimizer, patience=5, eps=1e-8)

Losses, AUCs, bestLossEpoch, bestAUCsEpoch = fit_(model=model, optimizer=optimizer, scheduler=None, epochs=cfg.epochs,
                                                  trainloader=tr_data, validloader=va_data,
                                                  criterion=nn.BCEWithLogitsLoss(), device=cfg.device)

TL = []
VL = []
TA = []
VA = []

for i in range(len(Losses)):
    TL.append(Losses[i]["train"])
    VL.append(Losses[i]["valid"])
    TA.append(AUCs[i]["train"])
    VA.append(AUCs[i]["valid"])

plt.figure()
plt.plot([i+1 for i in range(len(TL))], TL, "r", label="Train Loss")
plt.plot([i+1 for i in range(len(VL))], VL, "b--", label="Valid Loss")
plt.xlabel("Epochs -->")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.plot([i+1 for i in range(len(TA))], TA, "r", label="Train AUC")
plt.plot([i+1 for i in range(len(VA))], VA, "b--", label="Valid AUC")
plt.xlabel("Epochs -->")
plt.ylabel("AUC")
plt.legend()
plt.grid()
plt.show()

In [ ]:
ss = pd.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv")

y_pred_bl = predict_(model=model, dataloader=ts_data, device=cfg.device, mode="test", path="./Epoch_{}.pt".format(bestLossEpoch))
# y_pred_ba = predict_(model=model, dataloader=ts_data, device=cfg.device, mode="test", path="./Epoch_{}.pt".format(bestAUCsEpoch))

ss["target"] = y_pred_bl
ss.to_csv("./submission.csv", index=False)
ss.head(5)